## NJGPA Import Script
Creating a script that creates a template for NJGPA testing imports

In [1]:
import pandas as pd
import datetime as dt
from datetime import date

In [2]:
# importing data
df = pd.read_csv (r"C:\Users\togarro\Downloads\NJGPA_Fall_26.csv", parse_dates = ['Unit1OnlineTestStartDateTime'])

In [3]:
df.shape

(78, 205)

In [4]:
# modifying date format for TestDate column
df['TestDate'] = df['Unit1OnlineTestStartDateTime'].dt.strftime('%m%d%Y')

In [5]:
# function to return entity
def entity(row):
    if row['TestingSchoolCode'] == 50:
        return '010'
df['Testing School'] = df.apply(entity, axis = 1)

In [6]:
df['Test'] = 'NJGPA'

In [7]:
# creating subject specific dataframes
math = df[df['TestCode']=='MATGP']
math.rename(columns = {'TestCode':'MATTest','TestScaleScore':'MATGP','TestPerformanceLevel':'MATLevel'},inplace = True)

ela = df[df['TestCode']=='ELAGP']
ela.rename(columns = {'TestCode':'ELATest','TestScaleScore':'ELAGP','TestPerformanceLevel':'ELALevel'},inplace = True)

C:\Users\togarro\AppData\Local\Temp\ipykernel_17812\3844467848.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  math.rename(columns = {'TestCode':'MATTest','TestScaleScore':'MATGP','TestPerformanceLevel':'MATLevel'},inplace = True)
C:\Users\togarro\AppData\Local\Temp\ipykernel_17812\3844467848.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ela.rename(columns = {'TestCode':'ELATest','TestScaleScore':'ELAGP','TestPerformanceLevel':'ELALevel'},inplace = True)


In [8]:
#segmenting data based on subject
pii = df[['StateStudentIdentifier','LocalStudentIdentifier','LastOrSurname','FirstName','GradeLevelWhenAssessed','Test','TestDate']]
ela = ela[['StateStudentIdentifier','ELATest','ELAGP','ELALevel','Testing School']]#--> ela test data
math = math[['StateStudentIdentifier','MATTest','MATGP','MATLevel']] #--mat testing data

In [9]:
# creating the NJGPA_Upload_File
NJGPA_Upload_File = pd.DataFrame({'StateStudentIdentifier':df['StateStudentIdentifier'].drop_duplicates()})
NJGPA_Upload_File = NJGPA_Upload_File.merge(pii, on = 'StateStudentIdentifier', how = 'left')

In [10]:
# Merging files on the StateStudentIdentifier primary key
NJGPA_Upload_File  = NJGPA_Upload_File.merge(ela, on = 'StateStudentIdentifier', how = 'left').merge(math, on = 'StateStudentIdentifier', how = 'left')

In [11]:
NJGPA_Upload_File['Testing School'] = NJGPA_Upload_File['Testing School'].fillna('010')
NJGPA_Upload_File['Testing School'] = NJGPA_Upload_File['Testing School'].astype('string')

In [12]:
NJGPA_Upload_File.columns

Index(['StateStudentIdentifier', 'LocalStudentIdentifier', 'LastOrSurname',
       'FirstName', 'GradeLevelWhenAssessed', 'Test', 'TestDate', 'ELATest',
       'ELAGP', 'ELALevel', 'Testing School', 'MATTest', 'MATGP', 'MATLevel'],
      dtype='object')

In [13]:
NJGPA_Upload_File = NJGPA_Upload_File.drop_duplicates()

In [14]:
date_today = date.today()

In [15]:
NJGPA_Upload_File.to_csv(f'NJGPA_Upload_File_{date_today}.csv', index = False)